# SURFSPECALB Plots

[Click here](https://www.arm.gov/capabilities/vaps/surfspecalb) for more information about this vap.

In [ ]:
%matplotlib widget
import ipywidgets as widgets

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from datetime import datetime

import act
import xarray as xr

In [ ]:
# Data archive directory
DATA_DIR = r'/data/archive/'

In [ ]:
# Datastream info
DATASTREAM_NAME = 'surfspecalb1mlawer'
DATA_LEVEL = 'c1'
LOCATIONS = [{'end_date': '2020-10-06', 'facility': 'C1', 'site': 'nsa', 'start_date': '1998-04-07'}, {'end_date': '2020-08-05', 'facility': 'C1', 'site': 'sgp', 'start_date': '2004-01-01'}]

## Define site, facility, and date ranges

In [ ]:
print("The following locations and date ranges are available for this VAP:")
display(pd.DataFrame(LOCATIONS, columns=['site', 'facility', 'start_date', 'end_date']))

In [ ]:
# Define site, facility, and date range below:
site_facility = ( 'nsa', 'C1' )

# Date range. Format: YYYY-MM-DD
date_start = '2020-09-30'
date_end = '2020-10-05'

## Load data files
Loads data files from /data/archive/

In [ ]:
# Compile list of files
site, facility = site_facility
d_date_start = datetime.strptime(date_start, '%Y-%m-%d')
d_date_end = datetime.strptime(date_end, '%Y-%m-%d')
dir_path = os.path.join(DATA_DIR + site, site + DATASTREAM_NAME + facility + r'.' + DATA_LEVEL )
files_list  = []
for f in os.listdir(dir_path):
    file_date_str = f.split(r'.')[2]
    try:
        file_date = datetime.strptime(file_date_str, '%Y%m%d')
    except Exception:
        continue 
    if d_date_start <= file_date and d_date_end >= file_date:
        files_list.append(os.path.join(dir_path, f))

# Load files as a single dataset
ds = act.io.armfiles.read_netcdf(files_list)
ds.clean.cleanup()
print(f'{len(files_list)} files loaded')
ds

## Plot time series data

In [ ]:
variables_to_plot = ['be_surface_albedo_mfr_broadband_10m', 'be_surface_albedo_mfr_narrowband_10m', 'be_surface_albedo_psp_broadband_10m']

In [ ]:
ts_display = act.plotting.TimeSeriesDisplay(ds)
ts_display.add_subplots((len(variables_to_plot),), figsize = (15,4*len(variables_to_plot)))

for i,v in enumerate(variables_to_plot):
    ts_ax = ts_display.plot(v, subplot_index=(i,), set_title=ds.variables[v].attrs['long_name'])
    ts_ax.grid()

plt.show()

## Quality check plots

In [ ]:
variable = variables_to_plot[0]

In [ ]:
if ('qc_' + variable) in ds.variables:

    # Plot
    qc_display = act.plotting.TimeSeriesDisplay(ds)
    qc_display.add_subplots((2,), figsize = (15,10))
    qc_ax = qc_display.plot(variable, subplot_index=(0,), set_title="QC results on field: " + ds.variables[variable].attrs['long_name'])
    qc_ax.grid()
    qc_display.qc_flag_block_plot(variable, subplot_index=(1,))

    plt.show()

## Field selection dropdown

In [ ]:
plt.ioff()

available_variables = [v for v in ds.variables if not('time' in v or v.startswith('qc_') or v.startswith('source_')) and 'long_name' in ds.variables[v].attrs]
variable = available_variables[-1]

dropdown = widgets.Dropdown(
    options = [(ds.variables[v].attrs['long_name'], v) for v in available_variables],
    value= variable,
    description='Field:',
    disabled=False,
)

dropdown.layout.margin = '0px 30% 0px 20%'
dropdown.layout.width = '50%'

i_display = act.plotting.TimeSeriesDisplay(ds)
i_display.add_subplots((1,), figsize = (15,5))
i_ax = i_display.plot(variable, subplot_index=(0,), set_title=ds.variables[variable].attrs['long_name'])
i_ax.grid()

i_fig = i_display.fig

def update_plot(change):
    i_ax.cla()
    i_ax_new = i_display.plot(change.new, subplot_index=(0,), set_title=ds.variables[change.new].attrs['long_name'])
    i_ax_new.grid()
    i_fig.canvas.draw()
    i_fig.canvas.flush_events()

dropdown.observe(update_plot, names='value')

widgets.AppLayout(
    header=dropdown,
    center=i_fig.canvas,
    pane_heights=[1, 6,1]
)

## Surface Albedo

In [ ]:


sa_var = 'surface_albedo_mfr_narrowband_10m'

sa_display = act.plotting.TimeSeriesDisplay(ds, subplot_shape=(1,), figsize=(15,5))

sa_ax = sa_display.plot(sa_var, force_line_plot=True, labels=True, set_title=ds.variables[sa_var].attrs['long_name'])
sa_ax.grid()

plt.show()